# SUDOKU

but this time we solve it with a sat solver

In [20]:
from z3 import *
import time
import numpy as np

Let's dive into the idea. What are the constraints? How do we define our domain? 
We will use a 9 x 9 x 9 boolean space representation of the grid. Every cell will be represented as a vectors. 
In that vector every value is 0 excepted the i-th value that will be 1 and represent the actual value of the cell.

Let's make the grid.

In [32]:
I = [i for i in range (9)] # rows
J = [j for j in range (9)] # columns
V = [v for v in range (9)] # values

In [ ]:

X = [ [ Int(f"x_{i+1}_{j+1}") for j in range(9) ]
      for i in range(9) ]

# each cell contains a value in {1, ..., 9}
cells_c  = [ And(1 <= X[i][j], X[i][j] <= 9)
             for i in range(9) for j in range(9) ]

# each row contains a digit at most once
rows_c   = [ Distinct(X[i]) for i in range(9) ]

# each column contains a digit at most once
cols_c   = [ Distinct([ X[i][j] for i in range(9) ])
             for j in range(9) ]

# each 3x3 square contains a digit at most once
sq_c     = [ Distinct([ X[3*i0 + i][3*j0 + j]
                        for i in range(3) for j in range(3) ])
             for i0 in range(3) for j0 in range(3) ]

sudoku_c = cells_c + rows_c + cols_c + sq_c

# sudoku instance, we use '0' for empty cells
instance = ((0,0,0,0,9,4,0,3,0),
            (0,0,0,5,1,0,0,0,7),
            (0,8,9,0,0,0,0,4,0),
            (0,0,0,0,0,0,2,0,8),
            (0,6,0,2,0,1,0,5,0),
            (1,0,2,0,0,0,0,0,0),
            (0,7,0,0,0,0,5,2,0),
            (9,0,0,0,6,5,0,0,0),
            (0,4,0,9,7,0,0,0,0))

instance_c = [ True if instance[i][j] == 0 else X[i][j] == instance[i][j]
               for i in range(9) for j in range(9) ]

s = Solver()
s.add(sudoku_c + instance_c)
if s.check() == sat:
    m = s.model()
    r = [ [ m.evaluate(X[i][j]) for j in range(9) ]
          for i in range(9) ]
    print_matrix(r)
else:
    print ("failed to solve")


[[7, 1, 5, 8, 9, 4, 6, 3, 2],
 [2, 3, 4, 5, 1, 6, 8, 9, 7],
 [6, 8, 9, 7, 2, 3, 1, 4, 5],
 [4, 9, 3, 6, 5, 7, 2, 1, 8],
 [8, 6, 7, 2, 3, 1, 9, 5, 4],
 [1, 5, 2, 4, 8, 9, 7, 6, 3],
 [3, 7, 6, 1, 4, 8, 5, 2, 9],
 [9, 2, 8, 3, 6, 5, 4, 7, 1],
 [5, 4, 1, 9, 7, 2, 3, 8, 6]]
0.06981396675109863 s


In [33]:
X = np.array([[[Bool(f'{i+1}_{j+1}_{v+1}') for v in V] for j in J] for i in I])

In [34]:
def exclusivity_given_a_set(set):
    constrs = []
    for s in set:
        for s1 in set:
            if s1 is not s:
                constrs.append(Or(And(s, Not(s1)), And(Not(s), s1)))
    return constrs

In [ ]:
cells_c = [exclusivity_given_a_set(X[i:i+1, j:j+1, :].flatten()) for i in I for j in J] # only one value is True for every cell

In [49]:
rows_c = [exclusivity_given_a_set(X[:, j:j+1, v:v+1].flatten()) for j in J for v in V] # only one value is True for every row

In [46]:
cols_c = [exclusivity_given_a_set(X[i:i+1, :, v:v+1].flatten()) for i in I for v in V] # only one value is True for every columns

In [48]:
sqr_c = [exclusivity_given_a_set(X[i:i+1, j:j+1, v:v+1].flatten()) for i in [0, 3, 6] for j in [0, 3, 6] for v in V] # only one value is True for every square

In [50]:
sudoku_c = cells_c + rows_c + cols_c + sqr_c

In [ ]:
# sudoku instance, we use '0' for empty cells
instance = ((0,0,0,0,9,4,0,3,0),
            (0,0,0,5,1,0,0,0,7),
            (0,8,9,0,0,0,0,4,0),
            (0,0,0,0,0,0,2,0,8),
            (0,6,0,2,0,1,0,5,0),
            (1,0,2,0,0,0,0,0,0),
            (0,7,0,0,0,0,5,2,0),
            (9,0,0,0,6,5,0,0,0),
            (0,4,0,9,7,0,0,0,0))